In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from transformers import BertTokenizer, BertModel, get_linear_schedule_with_warmup
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import f1_score, roc_auc_score
import logging
from tqdm import tqdm  # импорт прогресс-бара

# Настройка логирования
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# 1. Загрузка данных
try:
    data = pd.read_csv('processed_data.csv')  # путь к вашему файлу с данными
    logging.info("Данные успешно загружены.")
except Exception as e:
    logging.error(f"Ошибка при загрузке данных: {e}")
    raise

# 3. Предобработка данных: приведение к нижнему регистру и очистка
categories = [
    'Вопрос решен',
    'Нравится качество выполнения заявки',
    'Нравится качество работы сотрудников',
    'Нравится скорость отработки заявок',
    'Понравилось выполнение заявки',
    'Другое'
]

try:
    labels = data[categories].values.astype(int)
except KeyError as e:
    logging.error(f"Некоторые категории не найдены в данных: {e}")
    raise

# 4. Разделение на обучающую и валидационную выборки с стратификацией по первому классу
try:
    stratify_labels = labels.argmax(axis=1)
    train_texts, val_texts, train_labels, val_labels = train_test_split(
        data['comment'].values,
        labels,
        test_size=0.2,
        random_state=42,
        stratify=stratify_labels
    )
    logging.info("Разделение данных выполнено успешно.")
except Exception as e:
    logging.error(f"Ошибка при разделении данных: {e}")
    raise

# 5. Токенизация и подготовка датасета с русской моделью BERT
model_name = 'DeepPavlov/rubert-base-cased'
tokenizer = BertTokenizer.from_pretrained(model_name)

class CommentsDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=256):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length= max_length
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text= str(self.texts[idx])
        label= self.labels[idx]
        try:
            encoding= self.tokenizer.encode_plus(
                text,
                add_special_tokens=True,
                max_length=self.max_length,
                padding='max_length',
                truncation=True,
                return_tensors='pt'
            )
            input_ids= encoding['input_ids'].squeeze(0)
            attention_mask= encoding['attention_mask'].squeeze(0)
            return {
                'input_ids': input_ids,
                'attention_mask': attention_mask,
                'labels': torch.FloatTensor(label)
            }
        except Exception as e:
            logging.warning(f"Ошибка при токенизации текста: {e}")
            return {
                'input_ids': torch.zeros(self.max_length,dtype=torch.long),
                'attention_mask': torch.zeros(self.max_length,dtype=torch.long),
                'labels': torch.FloatTensor(label)
            }

train_dataset = CommentsDataset(train_texts, train_labels, tokenizer)
val_dataset = CommentsDataset(val_texts, val_labels, tokenizer)

class_counts= train_labels.sum(axis=0)
total_samples= len(train_labels)

epsilon=1e-6
class_weights = (total_samples - class_counts + epsilon) / (class_counts + epsilon)
class_weights_tensor= torch.FloatTensor(class_weights)
logging.info(f"Веса классов: {class_weights}")

# 6. Создаем веса для балансировки (для использования в WeightedRandomSampler)
sample_weights=[]
for label in train_labels:
    class_indices= np.where(label==1)[0]
    weights_for_classes = class_weights[class_indices] if len(class_indices)>0 else np.array([1.0])
    sample_weight= np.min(weights_for_classes) 
    sample_weights.append(sample_weight)

sample_weights=np.array(sample_weights)

sampler= WeightedRandomSampler(weights= sample_weights, num_samples=len(sample_weights), replacement=True)

train_loader= DataLoader(train_dataset, batch_size=16, sampler=sampler)
val_loader= DataLoader(val_dataset, batch_size=16)

# 7. Модель с несколькими выходами (multi-label)
class BertMultiLabelClassifier(nn.Module):
    def __init__(self, dropout=0.3):
        super(BertMultiLabelClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(model_name)
        self.dropout = nn.Dropout(dropout)
        self.classifier = nn.Linear(self.bert.config.hidden_size, len(categories))
    
    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output  
        pooled_output= self.dropout(pooled_output)
        logits= self.classifier(pooled_output)  
        return logits

device= torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model= BertMultiLabelClassifier()
model.to(device)

criterion= nn.BCEWithLogitsLoss(pos_weight=class_weights_tensor)

optimizer= optim.AdamW(model.parameters(), lr=2e-5)

num_epochs=15  
total_steps= len(train_loader)*num_epochs

scheduler= get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=int(0.1*total_steps),
    num_training_steps=total_steps
)

best_f1_score_on_val = 0  # для сохранения лучшей модели

# Параметры ранней остановки (например: patience - сколько эпох ждать без улучшения)
patience = 3 
epochs_without_improvement = 0

# Инициализация переменной для лучшего значения F1 micro
best_f1_micro = 0

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    
    # Прогресс-бар по батчам с tqdm
    with tqdm(total=len(train_loader), desc=f"Epoch {epoch+1}/{num_epochs}", unit='batch') as pbar:
        for batch_idx, batch in enumerate(train_loader):
            try:
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['labels'].to(device)

                optimizer.zero_grad()
                outputs = model(input_ids=input_ids, attention_mask=attention_mask)
                loss = criterion(outputs, labels)

                # градиентный клиппинг для стабилизации обучения
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

                loss.backward()
                optimizer.step()
                scheduler.step()

                total_loss += loss.item()

            except Exception as e:
                logging.warning(f"Ошибка во время обучения батча {batch_idx}: {e}")

            pbar.update(1)  # обновляем прогресс-бар

    avg_loss = total_loss / len(train_loader)
    logging.info(f"Эпоха {epoch+1}/{num_epochs} - Средний Loss: {avg_loss:.4f}")

    # --- Валидация ---
    model.eval()
    
    all_preds=[]
    all_true=[]
    
    with torch.no_grad():
        for batch_idx,batch in enumerate(tqdm(val_loader,f"Validation Epoch {epoch+1}", unit='batch')):
            try:
                input_ids=batch['input_ids'].to(device)
                attention_mask=batch['attention_mask'].to(device)
                labels=batch['labels'].cpu().numpy()

                outputs=model(input_ids=input_ids, attention_mask=attention_mask)
                preds=torch.sigmoid(outputs).cpu().numpy()

                all_preds.extend(preds)
                all_true.extend(labels)

            except Exception as e:
                logging.warning(f"Ошибка при обработке батча {batch_idx}: {e}")

    true_labels=np.array(all_true) 
    preds_array=np.array(all_preds) 

    pred_labels=(preds_array>=0.5).astype(int)

    # Вычисляем F1-score на валидации после каждой эпохи
    try:
        f1_macro=f1_score(true_labels,pred_labels,average='macro')
        f1_micro=f1_score(true_labels,pred_labels,average='micro')  # добавляем расчет F1 micro
        
        # Проверка и сохранение модели по F1 macro
        if f1_macro > best_f1_score_on_val:
            best_f1_score_on_val=f1_macro
            epochs_without_improvement=0
            
            torch.save(model.state_dict(),'best_bert_multilabel.pth')
            logging.info(f"Новая лучшая модель сохранена по F1 macro: {f1_macro:.4f}")
        
        else:
            epochs_without_improvement+=1

        # Проверка и сохранение модели по F1 micro
        if f1_micro > best_f1_micro:
            best_f1_micro=f1_micro
            # Можно сохранить отдельную модель или перезаписать ту же
            torch.save(model.state_dict(),'best_bert_multilabel_f1micro.pth')
            logging.info(f"Новая лучшая модель по F1 micro сохранена: {f1_micro:.4f}")

        # Ранняя остановка при отсутствии улучшений по macro
        if epochs_without_improvement >= patience:
            logging.info("Ранняя остановка: достигнут patience без улучшения.")
            break
        
        print(f"\nЭпоха {epoch+1} - F1 macro: {f1_macro:.4f}, F1 micro: {f1_micro:.4f}")
        
        # Вывод по классам (по желанию)
        for i,cate in enumerate(categories):
            try:
                score=f1_score(true_labels[:,i],pred_labels[:,i])
                print(f"F1-score для '{cate}': {score:.4f}")
            except Exception as e:
                logging.warning(f"Ошибка при вычислении F1 для '{cate}': {e}")
        
    except Exception as e:
        logging.error(f"Ошибка во время оценки или обучения: {e}")

2025-05-21 00:27:17,471 - INFO - Данные успешно загружены.
2025-05-21 00:27:17,475 - INFO - Разделение данных выполнено успешно.
2025-05-21 00:27:17,858 - INFO - Веса классов: [ 4.35071088 10.06862736  2.85324231  1.52572707  6.14556959  3.78389829]
Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model


Эпоха 1 - F1 macro: 0.3250, F1 micro: 0.3163
F1-score для 'Вопрос решен': 0.3217
F1-score для 'Нравится качество выполнения заявки': 0.2740
F1-score для 'Нравится качество работы сотрудников': 0.5664
F1-score для 'Нравится скорость отработки заявок': 0.0000
F1-score для 'Понравилось выполнение заявки': 0.2038
F1-score для 'Другое': 0.5843


Epoch 2/15: 100%|██████████| 71/71 [13:10<00:00, 11.13s/batch]
2025-05-21 00:54:46,610 - INFO - Эпоха 2/15 - Средний Loss: 0.8492
Validation Epoch 2: 100%|██████████| 18/18 [00:42<00:00,  2.34s/batch]
2025-05-21 00:55:32,125 - INFO - Новая лучшая модель сохранена по F1 macro: 0.5885
2025-05-21 00:55:35,780 - INFO - Новая лучшая модель по F1 micro сохранена: 0.6029



Эпоха 2 - F1 macro: 0.5885, F1 micro: 0.6029
F1-score для 'Вопрос решен': 0.4476
F1-score для 'Нравится качество выполнения заявки': 0.2485
F1-score для 'Нравится качество работы сотрудников': 0.8466
F1-score для 'Нравится скорость отработки заявок': 0.8611
F1-score для 'Понравилось выполнение заявки': 0.3521
F1-score для 'Другое': 0.7752


Epoch 3/15: 100%|██████████| 71/71 [3:06:00<00:00, 157.19s/batch]    
2025-05-21 04:01:36,236 - INFO - Эпоха 3/15 - Средний Loss: 0.6064
Validation Epoch 3: 100%|██████████| 18/18 [00:41<00:00,  2.32s/batch]
2025-05-21 04:02:21,472 - INFO - Новая лучшая модель сохранена по F1 macro: 0.6123
2025-05-21 04:02:25,289 - INFO - Новая лучшая модель по F1 micro сохранена: 0.6590


...

Epoch 9/15: 100%|██████████| 71/71 [13:22<00:00, 11.31s/batch]
2025-05-21 09:45:15,006 - INFO - Эпоха 9/15 - Средний Loss: 0.2183
Validation Epoch 9: 100%|██████████| 18/18 [00:42<00:00,  2.37s/batch]
2025-05-21 09:46:01,128 - INFO - Новая лучшая модель по F1 micro сохранена: 0.7367



Эпоха 9 - F1 macro: 0.6711, F1 micro: 0.7367
F1-score для 'Вопрос решен': 0.4950
F1-score для 'Нравится качество выполнения заявки': 0.5294
F1-score для 'Нравится качество работы сотрудников': 0.8931
F1-score для 'Нравится скорость отработки заявок': 0.8987
F1-score для 'Понравилось выполнение заявки': 0.4301
F1-score для 'Другое': 0.7805


Epoch 10/15: 100%|██████████| 71/71 [13:19<00:00, 11.26s/batch]
2025-05-21 09:59:20,761 - INFO - Эпоха 10/15 - Средний Loss: 0.1685
Validation Epoch 10: 100%|██████████| 18/18 [00:42<00:00,  2.35s/batch]



Эпоха 10 - F1 macro: 0.6662, F1 micro: 0.7228
F1-score для 'Вопрос решен': 0.5000
F1-score для 'Нравится качество выполнения заявки': 0.6071
F1-score для 'Нравится качество работы сотрудников': 0.8931
F1-score для 'Нравится скорость отработки заявок': 0.8969
F1-score для 'Понравилось выполнение заявки': 0.3564
F1-score для 'Другое': 0.7438


Epoch 11/15: 100%|██████████| 71/71 [13:20<00:00, 11.27s/batch]
2025-05-21 10:13:23,213 - INFO - Эпоха 11/15 - Средний Loss: 0.1531
Validation Epoch 11: 100%|██████████| 18/18 [00:42<00:00,  2.36s/batch]
2025-05-21 10:14:05,654 - INFO - Ранняя остановка: достигнут patience без улучшения.
2025-05-21 10:14:06,057 - INFO - Загружена лучшая модель по F1 macro.
2025-05-21 10:14:06,058 - WARNING - Не удалось загрузить модель по micro: [Errno 2] No such file or directory: 'best_bert_multilabel_f2micro.pth'


In [46]:
# После завершения обучения можно загрузить лучшую модель по macro или по micro
# try:
#     model.load_state_dict(torch.load('best_bert_multilabel.pth'))
#     logging.info("Загружена лучшая модель по F1 macro.")
# except Exception as e:
#     logging.warning(f"Не удалось загрузить модель по macro: {e}")

# Или загрузить модель с наилучшим показателем по micro (если нужно)
try:
    model.load_state_dict(torch.load('best_bert_multilabel_f1micro.pth'))
    logging.info("Модель загружена.")
except Exception as e:
    logging.warning(f"Не удалось загрузить модель: {e}")

2025-05-21 12:00:43,841 - INFO - Модель загружена.


In [47]:
# Оценка модели на валидационной выборке с обработкой ошибок и логированием
model.eval()

all_preds = []
all_true = []

try:
    with torch.no_grad():
        for batch_idx, batch in enumerate(val_loader):
            try:
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['labels'].cpu().numpy()

                outputs = model(input_ids=input_ids, attention_mask=attention_mask)
                preds = torch.sigmoid(outputs).cpu().numpy()

                all_preds.extend(preds)
                all_true.extend(labels)

            except Exception as e:
                logging.warning(f"Ошибка при обработке батча {batch_idx}: {e}")
except Exception as e:
    logging.error(f"Ошибка во время оценки: {e}")

true_labels = np.array(all_true)  # shape: (num_samples,num_classes)
preds_array = np.array(all_preds)

pred_labels = (preds_array >= 0.5).astype(int)

try:
    f1_macro = f1_score(true_labels, pred_labels, average='macro')
    f1_micro = f1_score(true_labels, pred_labels, average='micro')
except Exception as e:
    logging.error(f"Ошибка при вычислении F1-score: {e}")

# Расчет по классам
f1_class_scores = []
for i, cate in enumerate(categories):
    try:
        score = f1_score(true_labels[:, i], pred_labels[:, i])
        f1_class_scores.append(score)
        print(f"F1-score для '{cate}': {score:.4f}")
    except Exception as e:
        logging.warning(f"Ошибка при вычислении F1 для '{cate}': {e}")

# Вывод средних значений F1
try:
    # Среднее по классам (macro уже есть как f1_macro)
    print(f"\nСредний F1-score (macro): {f1_macro:.4f}")
    print(f"F1-score (micro): {f1_micro:.4f}")
except NameError:
    print("Некорректные значения F1-score.")

print('--------------------------------------------------------------------------')

# ROC-AUC по классам с обработкой ошибок
roc_auc_scores = []
for i, cate in enumerate(categories):
    try:
        score = roc_auc_score(true_labels[:, i], preds_array[:, i])
        roc_auc_scores.append(score)
        print(f"ROC-AUC для '{cate}': {score:.4f}")
    except ValueError as e:
        roc_auc_scores.append(None)
        print(f"ROC-AUC для '{cate}': недоступен ({e})")
        
if any(score is not None for score in roc_auc_scores):
    valid_scores = [score for score in roc_auc_scores if score is not None]
    roc_auc_mean = np.mean(valid_scores) if valid_scores else None
else:
    roc_auc_mean = None

if roc_auc_mean is not None:
    print(f"\nСредний ROC-AUC по классам: {roc_auc_mean:.4f}")
else:
     print("\nНет доступных значений ROC-AUC для вычисления среднего.")

F1-score для 'Вопрос решен': 0.4950
F1-score для 'Нравится качество выполнения заявки': 0.5294
F1-score для 'Нравится качество работы сотрудников': 0.8931
F1-score для 'Нравится скорость отработки заявок': 0.8987
F1-score для 'Понравилось выполнение заявки': 0.4301
F1-score для 'Другое': 0.7805

Средний F1-score (macro): 0.6711
F1-score (micro): 0.7367
--------------------------------------------------------------------------
ROC-AUC для 'Вопрос решен': 0.7590
ROC-AUC для 'Нравится качество выполнения заявки': 0.8845
ROC-AUC для 'Нравится качество работы сотрудников': 0.9663
ROC-AUC для 'Нравится скорость отработки заявок': 0.9593
ROC-AUC для 'Понравилось выполнение заявки': 0.7998
ROC-AUC для 'Другое': 0.9196

Средний ROC-AUC по классам: 0.8814
